In [ ]:
!pip install pyLDAvis==3.3.1

In [ ]:
import nltk
import gensim
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn import svm
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora

df=pd.read_csv('data.csv')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:

data_remove_duplicatedContentUrl = df.drop_duplicates('Content_url')
print(len(data_remove_duplicatedContentUrl))

# reset index
data_remove_duplicatedContentUrl.reset_index(inplace=True,drop=True)

6859


In [ ]:

new_df = data_remove_duplicatedContentUrl.copy(deep=True) 
ps = PorterStemmer()

def nlp_processing(desc):
    # word  word_tokenize on description
    token_list1 = [] 
    wordpunkt_tkn = nltk.WordPunctTokenizer()
    tokens = wordpunkt_tkn.tokenize(desc)
    # tokens = nltk.word_tokenize(desc)
    
    #stop words 
    stop = set(stopwords.words('english'))
    for token in tokens:
        if token not in stop:
            token_list1.append(token)
        
    token_tag = nltk.pos_tag(token_list1)
    pos_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens1 = [x for x, tag in token_tag if tag in pos_tags]
    
    # stem words.
    stemmed_tokens = [ps.stem(x).lower() for x in filtered_tokens1]
    return set(stemmed_tokens)


new_df['des_word'] = new_df[new_df['Content_text'].notnull()]['Content_text'].map(nlp_processing)

In [ ]:


KeyTools = set(['rust', 'php', 'cntk','google+cloud', 'sql+server','amazon+web+services','lightgbm', 'kubeflow','unixlinux', 'postgressql', 'postgresql', 'postgres', 'hbase', 'dask', 'ruby', 'julia', 'tensor','bigquery', 'unix', 'react',
 'scikit', 'powerbi', 's3', 'ec2', 'lambda', 'ssrs', 'kubernetes', 'hana', 'spacy', 'django', 'sagemaker',
 'seaborn', 'mllib', 'github', 'git', 'elasticsearch', 'splunk', 'airflow', 'looker', 'rapidminer', 'birt', 'pentaho', 
 'jquery', 'nodejs', 'plotly', 'bokeh', 'xgboost', 'rstudio', 'shiny', 'dash', 'hadoop', 'mapreduce', 
 'hive', 'cognos', 'angular', 'nltk', 'flask', 'node', 'firebase', 'bigtable','dplyr','ggplot2','esquisse','bioconductor','shiny','lubridate','knitr','mlr','quanteda','dt','rcrawler','caret','rmarkdown',
 'leaflet','janitor','ggvis','plotly','rcharts','rbokeh','broom','stringr','magrittr','slidify','rvest',
 'rmysql','rsqlite','prophet','glmnet','text2vec','snowballc','quantmod','rstan','swirl','datasciencer','python', 'pytorch', 'sql', 'mxnet', 'mlflow', 'einstein', 'theano', 'pyspark', 'solr', 'mahout', 
 'cassandra', 'aws', 'powerpoint', 'spark', 'pig', 'sas', 'java', 'nosql', 'docker', 'salesforce', 'scala', 'r',
 'c', 'c++', 'net', 'tableau', 'pandas', 'scikitlearn', 'sklearn', 'matlab', 'scala', 'keras', 'tensorflow', 'clojure',
 'caffe', 'scipy', 'numpy', 'matplotlib', 'vba', 'spss', 'linux', 'azure', 'cloud', 'gcp', 'mongodb', 'mysql', 'oracle', 
 'redshift', 'snowflake', 'kafka', 'javascript', 'qlik', 'jupyter', 'perl'])

# Skills = set(['optimizing', 'prediction', 'forecasting',
#  'clustering', 'cluster', 'optimization', 'visualization', 'nlp', 'c#','regression', 'logistic', 'nn', 'cnn', 'glm','rnn', 'lstm', 'gbm', 'boosting', 'recurrent', 'convolutional', 'bayesian',
#  'bayes','random forest', 'natural language processing', 'machine learning', 'statistics', 'cleansing', 'chatbot', 'cleaning', 'blockchain', 'causality', 'correlation', 'bandit', 'anomaly', 'kpi',
#  'dashboard', 'geospatial', 'ocr', 'econometrics', 'pca', 'gis', 'svm', 'svd', 'tuning', 'hyperparameter', 'hypothesis',
#  'salesforcecom', 'segmentation', 'biostatistics', 'unsupervised', 'supervised', 'exploratory',
#  'recommender', 'recommendations', 'research', 'sequencing', 'probability', 'reinforcement', 'graph', 'bioinformatics',
#  'chi', 'knn', 'outlier', 'etl', 'normalization', 'classification','decision tree', 'deep learning', 'experimental design',
#  'time series', 'nearest neighbors', 'neural network', 'support vector machine', 'computer vision', 'machine vision', 'dimensionality reduction', 
#  'text analytics', 'power bi', 'a/b testing', 'ab testing', 'chat bot', 'data mining','vision'])

Skills = set(['nlp', 'cnn', 'rnn', 'lstm'])

degree  = set(['bachelor', 'master', 'doctor','ms','ph.d','phd'])



df_temp=new_df.copy(deep=True)

def add_feature_set(set_customer, df):
        result = set([ps.stem(x) for x in set_customer])
        list_result = []

        num_postings = len(df['Content_text'].index)

        # range of dataframe length
        for i in range(num_postings):
                desc_token = df.iloc[i,11]
                result_words = result.intersection(desc_token)
                list_result.append(result_words)


        for i in range(len(list_result)):

            if(list_result[i]==set()):
                continue
            else:
                for j in list_result[i]: 
                    if (j not in df.columns):
                        df[j] = 0 
                    df.loc[i,[j]] = 1

        return df

df_add_degree = add_feature_set(degree, df_temp)

NameError: name 'new_df' is not defined

In [ ]:
# LDA
new_df = data_remove_duplicatedContentUrl.copy(deep=True) 
df_temp2=new_df.copy(deep=True)

# Preprocessing - Create Term Frequency Matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Let's remove domain specific stop words like 'work'
from nltk.corpus import stopwords
stop = list(stopwords.words('english')) + ['work','experience','data','company','new','years','skills','team','teams','science','technology','us','including']

# LDA can only use raw term counts for LDA 
tf_vectorizer = CountVectorizer(min_df=5, stop_words=stop)
tf = tf_vectorizer.fit_transform(df_temp2['Content_text'])

# each feature is a word (bag of words)
# get_feature_names() gives all words
tf_feature_names = tf_vectorizer.get_feature_names()

# print(tf_feature_names[0:10])
# print(tf.shape)

#### 使用gensim
# Create LDA model using the same TF matrix generated from sklearn

# convert the gensim corpus from the sparse tf matrix
corpus = gensim.matutils.Sparse2Corpus(tf, documents_columns=False)

# create the mapping between id and words
id2word={idx:w for idx, w in enumerate(tf_vectorizer.get_feature_names())}

# create a gensim dictionary from the corpus
dictionary = corpora.Dictionary.from_corpus(corpus, id2word=id2word)

# Train LDA model
NUM_TOPICS = 4
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, \
                            id2word=id2word, \
                            iterations=30)
topics = ldamodel.print_topics(num_words=20)
for topic in topics:
    print(topic)
    print("\n")
    

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
(0, '0.006*"business" + 0.006*"research" + 0.005*"analysis" + 0.004*"analytics" + 0.004*"opportunity" + 0.004*"information" + 0.004*"development" + 0.004*"working" + 0.004*"health" + 0.004*"medical" + 0.004*"learning" + 0.004*"ability" + 0.004*"world" + 0.003*"support" + 0.003*"status" + 0.003*"design" + 0.003*"management" + 0.003*"knowledge" + 0.003*"solutions" + 0.003*"machine"')


(1, '0.011*"business" + 0.008*"product" + 0.006*"technical" + 0.005*"customer" + 0.005*"solutions" + 0.005*"engineering" + 0.005*"customers" + 0.005*"development" + 0.005*"ability" + 0.004*"status" + 0.004*"management" + 0.004*"design" + 0

In [ ]:
# visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary, sort_topics=True, R=20)
pyLDAvis.display(lda_display)

In [ ]:
# assign topic
# test_corpus = gensim.matutils.Sparse2Corpus(tf, documents_columns=False)
predict = ldamodel.get_document_topics(corpus)


df_temp3=new_df.copy(deep=True)

list_topics = []
for i in range(len(predict)):
    list_topics.append(predict[i][0][0]) 

df_temp3['topic_category'] = list_topics

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 0.7000995), (2, 0.20040146), (3, 0.09589622)],
 [(1, 0.6060809), (2, 0.39207456)],
 [(0, 0.06769992), (1, 0.36618018), (2, 0.098787546), (3, 0.46733233)],
 [(0, 0.06395269), (1, 0.07263825), (2, 0.02709523), (3, 0.83631384)],
 [(1, 0.07564), (2, 0.55475515), (3, 0.3690532)]]

In [ ]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn import svm
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df=pd.read_csv('data.csv')
df = df.drop_duplicates('Content_url')
df.reset_index(inplace=True,drop=True)

df_temp=df.copy(deep=True)

# salary

In [ ]:
df_temp=df_temp.replace(to_replace='None', value=np.nan)
df_temp=df_temp.dropna(subset=['Salary'])
df_temp.reset_index(inplace=True,drop=True)
df_temp['Salary'].replace(regex=True, inplace=True, to_replace="\n", value="")
df_temp['Salary'].replace(regex=True, inplace=True, to_replace=",", value="")
df_temp['Salary'].replace(regex=True, inplace=True, to_replace="\$", value="")

df_temp.loc[df_temp['Salary'].str.contains('year'), 'salary_type'] = 'year'
df_temp.loc[df_temp['Salary'].str.contains('month'), 'salary_type'] = 'month'
df_temp.loc[df_temp['Salary'].str.contains('week'), 'salary_type'] = 'week'
df_temp.loc[df_temp['Salary'].str.contains('day'), 'salary_type'] = 'day'
df_temp.loc[df_temp['Salary'].str.contains('hour'), 'salary_type'] = 'hour'

df_temp['Salary'].replace(r'[a-zA-Z%]', '', regex=True, inplace=True)
df_temp['salary_range'] = df_temp['Salary'].str.split('-')

def mean(df ):
    df_temp['lower_cut'] = df_temp['salary_range'].str[0].astype('float') 
    df_temp['upper_cut'] = df_temp['salary_range'].str[1].astype('float') 
    df_temp['SalaryMean'] = df_temp[['lower_cut','upper_cut']].mean(axis=1)
    
mean(df_temp)
df_temp.drop(['salary_range', 'lower_cut', 'upper_cut'], axis=1, inplace=True)

for index in range(0,df_temp.shape[0],1):
    data = df_temp.at[index,'salary_type']
    if data == 'month':
        df_temp.at[index,'SalaryMean'] = df_temp.at[index,'SalaryMean'].astype(float) * 12 
    elif data == 'week':
        df_temp.at[index,'SalaryMean'] = df_temp.at[index,'SalaryMean'].astype(float) * 52 
    elif data == 'day':
        df_temp.at[index,'SalaryMean'] = df_temp.at[index,'SalaryMean'].astype(float) * 260
    elif data == 'hour':
        df_temp.at[index,'SalaryMean'] = df_temp.at[index,'SalaryMean'].astype(float) * 2080
        
df_temp.drop(['salary_type'], axis=1, inplace=True)

In [ ]:
chart = sns.histplot(df_temp["SalaryMean"])
chart.axvline(df_temp["SalaryMean"].mean(), lw=2, color='black')
chart.set_title('Salary Mean across Indeed Jobs Data')

# one-hot encoding

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

df_temp=encode_and_bind(df_temp,'SearchRegion')
df_temp=encode_and_bind(df_temp,'SearchDomain')

In [ ]:
df_temp=df_temp.drop(columns=['CompanyName','CompanyLocation','Salary','Content_url','CompanyRating','Date','Extract_Date'])

# JobTitle

In [ ]:
df_temp['JobTitle']= df_temp['JobTitle'].str.lower()
df_temp['JobTitle']=df_temp['JobTitle'].str.replace('new','')
df_temp['JobTitle']=df_temp['JobTitle'].str.replace('_|-',' ')
df_temp['JobTitle']=df_temp['JobTitle'].str.replace('/',' ')
df_temp['JobTitle']=df_temp['JobTitle'].str.replace('  ',' ')
df_temp['JobTitle']=df_temp['JobTitle'].str.replace('sr.|sr|lead|head','senior')
df_temp['JobTitle']=df_temp['JobTitle'].str.replace('jr.|jr|fresher','junior')
df_temp['JobTitle']=df_temp['JobTitle'].str.replace('ml','machine learning')

In [ ]:
df_temp['Senior']=0
subs=['senior','manager','expert','scientist','specialist']
for idx, row in df_temp.iterrows():
    for sub in subs:
        if 'senior' in row['JobTitle']:
            df_temp.loc[idx,'Senior']=1
            break

In [ ]:
tfidf_vect = TfidfVectorizer(stop_words='english',min_df=5)
jt_tfidf= tfidf_vect.fit_transform(df_temp['JobTitle'])

temp=pd.DataFrame(jt_tfidf.todense(),columns=tfidf_vect.get_feature_names())
df_temp=pd.merge(df_temp, temp, left_index=True, right_index=True)

df_temp=df_temp.drop(columns=['JobTitle'])

# Degree

In [ ]:
ps = PorterStemmer()

def nlp_processing(desc):
    # word  word_tokenize on description
    token_list1 = [] 
    pattern=r'\w[\w\',-]*\w'  
    tokens=nltk.regexp_tokenize(desc, pattern)
    # tokens = nltk.word_tokenize(desc)
    
    #stop words 
    stop = set(stopwords.words('english'))
    for token in tokens:
        if token not in stop:
            token_list1.append(token)
        
    token_tag = nltk.pos_tag(token_list1)
    pos_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens1 = [x for x, tag in token_tag if tag in pos_tags]
    
    # stem words.
    stemmed_tokens = [ps.stem(x).lower() for x in filtered_tokens1]
    return set(stemmed_tokens)


df_temp['des_word'] = df_temp[df_temp['Content_text'].notnull()]['Content_text'].map(nlp_processing)
df_temp=df_temp.drop(columns=['Content_text'])

In [ ]:
KeyTools = set(['rust', 'php', 'cntk','google+cloud', 'sql+server','amazon+web+services','lightgbm', 'kubeflow','unixlinux', 'postgressql', 'postgresql', 'postgres', 'hbase', 'dask', 'ruby', 'julia', 'tensor','bigquery', 'unix', 'react',
 'scikit', 'powerbi', 's3', 'ec2', 'lambda', 'ssrs', 'kubernetes', 'hana', 'spacy', 'django', 'sagemaker',
 'seaborn', 'mllib', 'github', 'git', 'elasticsearch', 'splunk', 'airflow', 'looker', 'rapidminer', 'birt', 'pentaho', 
 'jquery', 'nodejs', 'plotly', 'bokeh', 'xgboost', 'rstudio', 'shiny', 'dash', 'hadoop', 'mapreduce', 
 'hive', 'cognos', 'angular', 'nltk', 'flask', 'node', 'firebase', 'bigtable','dplyr','ggplot2','esquisse','bioconductor','shiny','lubridate','knitr','mlr','quanteda','dt','rcrawler','caret','rmarkdown',
 'leaflet','janitor','ggvis','plotly','rcharts','rbokeh','broom','stringr','magrittr','slidify','rvest',
 'rmysql','rsqlite','prophet','glmnet','text2vec','snowballc','quantmod','rstan','swirl','datasciencer','python', 'pytorch', 'sql', 'mxnet', 'mlflow', 'einstein', 'theano', 'pyspark', 'solr', 'mahout', 
 'cassandra', 'aws', 'powerpoint', 'spark', 'pig', 'sas', 'java', 'nosql', 'docker', 'salesforce', 'scala', 'r',
 'c', 'c++', 'net', 'tableau', 'pandas', 'scikitlearn', 'sklearn', 'matlab', 'scala', 'keras', 'tensorflow', 'clojure',
 'caffe', 'scipy', 'numpy', 'matplotlib', 'vba', 'spss', 'linux', 'azure', 'cloud', 'gcp', 'mongodb', 'mysql', 'oracle', 
 'redshift', 'snowflake', 'kafka', 'javascript', 'qlik', 'jupyter', 'perl'])

# Skills = set(['optimizing', 'prediction', 'forecasting',
#  'clustering', 'cluster', 'optimization', 'visualization', 'nlp', 'c#','regression', 'logistic', 'nn', 'cnn', 'glm','rnn', 'lstm', 'gbm', 'boosting', 'recurrent', 'convolutional', 'bayesian',
#  'bayes','random forest', 'natural language processing', 'machine learning', 'statistics', 'cleansing', 'chatbot', 'cleaning', 'blockchain', 'causality', 'correlation', 'bandit', 'anomaly', 'kpi',
#  'dashboard', 'geospatial', 'ocr', 'econometrics', 'pca', 'gis', 'svm', 'svd', 'tuning', 'hyperparameter', 'hypothesis',
#  'salesforcecom', 'segmentation', 'biostatistics', 'unsupervised', 'supervised', 'exploratory',
#  'recommender', 'recommendations', 'research', 'sequencing', 'probability', 'reinforcement', 'graph', 'bioinformatics',
#  'chi', 'knn', 'outlier', 'etl', 'normalization', 'classification','decision tree', 'deep learning', 'experimental design',
#  'time series', 'nearest neighbors', 'neural network', 'support vector machine', 'computer vision', 'machine vision', 'dimensionality reduction', 
#  'text analytics', 'power bi', 'a/b testing', 'ab testing', 'chat bot', 'data mining','vision'])

Skills = set(['nlp', 'cnn', 'rnn', 'lstm'])

degree  = set(['bachelor', 'master', 'doctor','bs','b.s','ms','m.s','ph.d','phd'])

In [ ]:
df_temp=df_temp.dropna(subset=['des_word'])
df_temp.reset_index(inplace=True,drop=True)

In [ ]:
def add_feature_set(set_customer, df):
        result = set([ps.stem(x) for x in set_customer])
        list_result = []
        len(df['des_word'])
        for i in range(len(df['des_word'])):
            desc_token = df['des_word'][i]
            result_words = result.intersection(desc_token)
            list_result.append(result_words)

        for i in range(len(list_result)):
            if(list_result[i]==set()):
                continue
            else:
                for j in list_result[i]: 
                    if (j not in df.columns):
                        df[j] = 0 
                    df.loc[i,[j]] = 1 

        return df

df_temp = add_feature_set(degree, df_temp)

df_temp['BS']=df_temp['bs']|df_temp['bachelor']
df_temp.drop(['bs','bachelor'], axis=1, inplace=True)
df_temp['MS']=df_temp['ms']|df_temp['master']
df_temp.drop(['ms','master'], axis=1, inplace=True)
df_temp['PHD']=df_temp['phd']|df_temp['doctor']
df_temp.drop(['phd','doctor'], axis=1, inplace=True)

df_temp.drop(['des_word'], axis=1, inplace=True)

# Model

In [ ]:
#l=list(range(1,10))+list(range(100,103))
data_X=np.array(df_temp.iloc[:,1:])
data_y=np.array(df_temp.iloc[:,0]>df_temp['SalaryMean'].mean()).astype(int)
print(data_X.shape)

x_train,x_test,y_train,y_test=train_test_split(data_X,data_y,test_size=0.1,random_state=123)

In [ ]:
l=list(range(0,10))+list(range(100,103))
corr = df_temp.iloc[:,l].corr()
sns.heatmap(corr,  annot=True)

In [ ]:
svm_clf = svm.SVC()
svm_clf.fit(x_train,y_train)
pred_svm_train=svm_clf.predict(x_train)
pred_svm_test=svm_clf.predict(x_test)
print(accuracy_score(pred_svm_train,y_train))
print(accuracy_score(pred_svm_test,y_test))

In [ ]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(x_train,y_train)
pred_lgb_train=lgb_clf.predict(x_train)
pred_lgb_test=lgb_clf.predict(x_test)
print(accuracy_score(pred_lgb_train,y_train))
print(accuracy_score(pred_lgb_test,y_test))

In [ ]:
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(x_train,y_train)
pred_xgb_train=xgb_clf.predict(x_train)
pred_xgb_test=xgb_clf.predict(x_test)
print(accuracy_score(pred_xgb_train,y_train))
print(accuracy_score(pred_xgb_test,y_test))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4e77b477-2259-494e-aaf4-b532e5790ad6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>